In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

In [ ]:
def Datos(path):
    data = [[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return np.array(data)

def EffectiveMass(Corr,Nt):
    #We need the root of this function
    def f(x,Corr,t):
        return Corr[t]/Corr[(t+1)%Nt] - np.cosh(x * (t-Nt/2))/ np.cosh(x * ( (t+1)%Nt - Nt/2 ))
    meff = []
    for t in range(Nt):
        sol = optimize.root_scalar(f, args=(Corr,t), method='secant', x0=0.5, x1=1, xtol=1e-15, rtol=1e-15, maxiter=1000)
        if sol.converged == True:
            meff.append(sol.root)
        else:
            print("Solution not found for t={0}".format(t))
            meff.append(np.nan)
    return np.array(meff)
          

In [ ]:
beta, Nx, Nt = 2, 16, 16
if beta == 3:
    masses = [-0.5,-0.4,-0.3,-0.2,-0.15,-0.1,0.05,0.1,0.15]
elif beta == 2:
    masses = [-0.16]

plt.title(r'Correlation function, $N_x$={0}, $N_t={1}$, $\beta$={2}'.format(Nx,Nt,beta))
plt.ylabel(r'$|c(t)|$')
plt.xlabel(r"$t$")

for m0 in masses:
    path = 'confs/b{0}_{1}x{2}/2D_U1_Ns{1}_Nt{2}_b{0}_m{3}_corr.txt'.format(beta,Nx,Nt,str(m0).replace(".",""))
    t, Corr, dCorr = Datos(path)
    Corr = abs(Corr)
    plt.errorbar(t,Corr,yerr=dCorr,\
        fmt='*',markersize='5',elinewidth=0.5,solid_capstyle='projecting',label='$m_0=${0}'.format(m0), capsize=1.5)
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
for t in range(Nt):
    print('C({0})= {1} +- {2}'.format(t,Corr[t],dCorr[t]))

In [ ]:
plt.title(r'Effective mass, $N_x$={0}, $N_t={1}$, $\beta$={2}'.format(Nx,Nt,beta))
plt.ylabel(r'$m_{eff}(t)$')
plt.xlabel(r"$t$")

for m0 in masses:
    path = 'confs/b{0}_{1}x{2}/2D_U1_Ns{1}_Nt{2}_b{0}_m{3}_corr.txt'.format(beta,Nx,Nt,str(m0).replace(".",""))
    t, Corr, dCorr = Datos(path)
    Corr = abs(Corr)
    meff = EffectiveMass(Corr,Nt)
    plt.errorbar(t,meff,yerr=meff*0,\
        fmt='*',markersize='5',elinewidth=0.5,solid_capstyle='projecting',label='$m_0=${0}'.format(m0), capsize=1.5)
#plt.yscale('log')
plt.legend()
plt.show()
print(np.mean(meff[2:12]))

In [ ]:
from scipy.optimize import curve_fit
def func(x, a, b):
    return a*np.exp((Nt/2-x)*b)

popt, pcov = curve_fit(func, np.arange(Nt)[4:8], Corr[4:8],p0=[0.1,0.1])
print(popt)
plt.plot(np.arange(Nt), func(np.arange(Nt), *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f'% tuple(popt))
plt.scatter(np.arange(Nt),Corr)
plt.yscale('log')

In [ ]:
kappa = 0.2680
mass = 1/(2*kappa)-2
print(mass)

### Using explicitly the Dirac Matrix

In [ ]:
from scipy.sparse import csc_array
from scipy.sparse.linalg import bicgstab

In [ ]:
def DatosMat(path):
    data = [[],[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

In [ ]:
Nx, Nt = 16, 16
num_configs = 1000
dim = Nx*Nt*2
print(dim)
D = np.zeros((dim,dim),dtype=complex)
Dinv = np.zeros((dim,2),dtype=complex)
CorrMat = np.zeros((Nt,num_configs))
Corr = np.zeros(Nt)
for nconf in range(num_configs):
    row, col, re, im = DatosMat("confs/D{0}.dat".format(nconf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    print("Conf {0}".format(nconf))
    for i in range(2):
        can_vec = np.eye(1,dim,i).reshape(dim)
        Dinvcol, exit_code = bicgstab(csc_array(D), can_vec, x0=can_vec,maxiter=1000, rtol=1e-10)
        if exit_code != 0:
            print("convergence failed")
        Dinv[:,i] = Dinvcol
    for t in range(Nt):
        for alf in range(2):
            for x in range(Nx):
                for bet in range(2):
                    indx = 2*(x*Nt + t)+alf 
                    #the indx for bet is just bet
                    CorrMat[t,nconf] += np.real(Dinv[indx,bet] * np.conjugate(Dinv[indx,bet]))
        CorrMat[t,nconf] /= np.sqrt(Nx)

In [ ]:
for t in range(Nt):
    Corr[t] = np.mean(CorrMat[t])
    dCorr[t] = np.std(CorrMat[t])/np.sqrt(num_configs)

In [ ]:
beta, Nx, Nt = 2, 16, 16
m0 = -0.16

plt.title(r'Correlation function, $N_x$={0}, $N_t={1}$, $\beta$={2}'.format(Nx,Nt,beta))
plt.ylabel(r'$|c(t)|$')
plt.xlabel(r"$t$")

plt.errorbar(np.arange(Nt),Corr,yerr=dCorr,\
    fmt='*',markersize='5',elinewidth=0.5,solid_capstyle='projecting',label='$m_0=${0} python'.format(m0), capsize=1.5)

path = 'confs/b{0}_{1}x{2}/2D_U1_Ns{1}_Nt{2}_b{0}_m{3}_corr.txt'.format(beta,Nx,Nt,str(m0).replace(".",""))
t, data, ddata = Datos(path)
data = abs(data)
plt.errorbar(t,data,yerr=ddata,\
    fmt='*',markersize='5',elinewidth=0.5,solid_capstyle='projecting',label='$m_0=${0}'.format(m0), capsize=1.5)
plt.yscale('log')
plt.legend()
plt.show()
for i in np.arange(Nt):
    print(Corr[i],data[i])

In [ ]:
plt.title(r'Effective mass, $N_x$={0}, $N_t={1}$, $\beta$={2}'.format(Nx,Nt,beta))
plt.ylabel(r'$m_{eff}(t)$')
plt.xlabel(r"$t$")
meff = EffectiveMass(Corr,Nt)
plt.errorbar(t,meff,yerr=meff*0,\
    fmt='*',markersize='5',elinewidth=0.5,solid_capstyle='projecting',label='$m_0=${0}'.format(m0), capsize=1.5)
#plt.yscale('log')
plt.legend()
plt.show()
print(np.mean(meff[2:12]))

In [ ]:
def func(x, a, b):
    return a*np.exp((Nt/2-x)*b)

popt, pcov = curve_fit(func, np.arange(Nt)[4:8], Corr[4:8],p0=[0.1,0.1])
print(popt)
plt.plot(np.arange(Nt), func(np.arange(Nt), *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f'% tuple(popt))
plt.scatter(np.arange(Nt),Corr)
plt.yscale('log')